In [2]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)


In [3]:
#config file 불러오기
# cfg = Config.fromfile('/opt/ml/CustomPipeline/baseline/mmdetection/configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_1x_coco.py')
cfg = Config.fromfile('/opt/ml/CustomPipeline/baseline/mmdetection/configs/detr/detr_r50_8x2_150e_coco.py')

classes = ("General trash","Paper","Papaer pack","Metal","Glass",
           "Plastic","Styrofoam","Plastic bag","Battery","Clothing")

#dataset 바꾸기
root = "/opt/ml/detection/dataset/"
cfg.data.train.classes = classes
cfg.data.train.img_prefix  = root
cfg.data.train.ann_file = root + 'train.json'
# cfg.data.train.pipeline[2]['img_scale'] = (1024,1024) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix  = root
cfg.data.val.ann_file = root + 'test.json'
# cfg.data.val.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix  = root
cfg.data.test.ann_file = root + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 4
cfg.seed= 42
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/detr_trash'

# cfg.model.roi_head.bbox_head.num_classes = 11
cfg.model.bbox_head.num_classes = 10
cfg.optimizer_config.grad_clip = dict(max_norm=35,norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3,interval=1)

In [4]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model)
model.init_weights()

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!


2022-03-27 08:11:09,884 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2022-03-27 08:11:09,885 - mmcv - INFO - load model from: torchvision://resnet50
2022-03-27 08:11:09,886 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2022-03-27 08:11:10,095 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-03-27 08:11:10,232 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from torchvision://resnet50 
 
2022-03-27 08:11:10,234 - mmcv - INFO - 
backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from torchvision://resnet50 
 
2022-03-27 08:11:10,234 - mmcv - INFO - 
backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from torchvision://resnet50 
 
2022-03-27 08:11:10,235 - mmcv - INFO - 
backbone.layer1.0.conv1.weight - torch.Size([64, 64, 1, 1]): 
PretrainedInit:

In [5]:
datasets

[
 CocoDataset Train dataset with number of images 4693, and instance counts: 
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | category          | count | category      | count | category        | count | category    | count | category     | count |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Papaer pack] | 936   | 3 [Metal]   | 982   | 4 [Glass]    | 2943  |
 | 5 [Plastic]       | 1263  | 6 [Styrofoam] | 5178  | 7 [Plastic bag] | 159   | 8 [Battery] | 468   | 9 [Clothing] | 0     |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+]

In [6]:
train_detector(model,datasets[0],cfg,distributed=False,validate=True)


2022-03-26 18:09:22,945 - mmdet - INFO - Start running, host: root@9075f89f9a55, work_dir: /opt/ml/CustomPipeline/MyModel/work_dirs/detr_trash
2022-03-26 18:09:22,946 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) Opti

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/opt/ml/detection/baseline/mmdetection/mmdet/models/losses/cross_entropy_loss.py:239: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  self.class_weight, device=cls_score.device)
2022-03-26 18:09:44,440 - mmdet - INFO - Epoch [1][50/1174]	lr: 1.000e-04, eta: 21:00:47, time: 0.430, data_time: 0.056, memory: 6414, loss_cls: 1.2723, loss_bbox: 3.2939, loss_iou: 2.0731, d0.loss_cls: 1.2485, d0.loss_bbox: 3.2975, d0.loss_iou: 2.0651, d1.loss_cls: 1.2604, d1.loss_bbox: 3.2869, d1.loss_iou: 2.0797, d2.loss_cls: 1.2594, d2.loss_bbox: 3.3043, d2.loss_iou: 2.0816, d3.loss_cls: 1.2654, d3.loss_bbox: 3.3060, d3.loss_iou: 2.0791, d4.loss_cls: 1.2536, d4.loss_bbox: 3.3043, d4.loss_iou: 2.0827, loss: 39.8137, grad_norm: 123.8499
2022-03-26 18:10:02,426 - mmdet - INFO - Epoch [1][100/1174]	lr: 1.000e-04, eta: 19:17:47, time: 0.360, data_time: 0.008, 

KeyboardInterrupt: 

In [24]:
model

DETR(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0):

In [14]:
dataset = [build_dataset(cfg.data.test)]
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [26]:
# epoch = 'latest'
# checkpoint path
# checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')
model = build_detector(cfg.model) # build detector
# checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

# model.CLASSES = dataset.CLASSES
# model = MMDataParallel(model.cuda(), device_ids=[0])

AssertionError: DETR: DETRHead: Expected class_weight to have type float. Found <class 'torch.Tensor'>.